# Team Project - Anomaly Detection

In [1]:
# Libraries
import pandas as pd
import numpy as np

# Wrangling
import acquire as a
import prepare as p

# Exploration/Visualizing
import matplotlib.pyplot as plt
import seaborn as sns
import explore as e
from IPython.core import display as ICD

from env import get_db_url


## Acquire
- Will acquire a dataset to use in answering our questions by doing the following.
    - Acquire a dataset from a SQL server database 
    - Save it locally as a .csv file
    - Convert the .csv file as a dataframe using Pandas

In [2]:
# returns a dataframe after acquiring curriculum log data from SQL database
df = a.get_log_data()

In [3]:
# returns information on number of non-nulls for each column and their datatype
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 900223 entries, 0 to 900222
Data columns (total 15 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   date        900223 non-null  object 
 1   time        900223 non-null  object 
 2   path        900222 non-null  object 
 3   user_id     900223 non-null  int64  
 4   cohort_id   847330 non-null  float64
 5   ip          900223 non-null  object 
 6   id          847330 non-null  float64
 7   name        847330 non-null  object 
 8   slack       847330 non-null  object 
 9   start_date  847330 non-null  object 
 10  end_date    847330 non-null  object 
 11  created_at  847330 non-null  object 
 12  updated_at  847330 non-null  object 
 13  deleted_at  0 non-null       object 
 14  program_id  847330 non-null  float64
dtypes: float64(3), int64(1), object(11)
memory usage: 103.0+ MB


In [4]:
# returns total number of sums for each column
df.isna().sum()

date               0
time               0
path               1
user_id            0
cohort_id      52893
ip                 0
id             52893
name           52893
slack          52893
start_date     52893
end_date       52893
created_at     52893
updated_at     52893
deleted_at    900223
program_id     52893
dtype: int64

- Total rows = 900223
- Deleted_at = 100% nulls
- Most columns have 52,893 nulls

## Prepare
 - Will prepare the dataframe to best answer the question in Explore section by doing the following
     - Dropping all rows with nulls
     - Dropping unnecessary columns
     - Converting columns to useful datatypes
     - Renaming columns for clarity

In [5]:
# shows preview the top of the dataframe
df.head(3)

,date,time,path,user_id,cohort_id,ip,id,name,slack,start_date,end_date,created_at,updated_at,deleted_at,program_id
0,2018-01-26,09:55:03,/,1,8.0,97.105.19.61,8.0,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,None,1.0
1,2018-01-26,09:56:02,java-ii,1,8.0,97.105.19.61,8.0,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,None,1.0
2,2018-01-26,09:56:05,java-ii/object-oriented-programming,1,8.0,97.105.19.61,8.0,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,None,1.0


In [6]:
# drops all rows with nulls; drops the columns 'slack', 'id', and 'deleted_at'
# ; concatenates 'date' and 'time' columns to single 'date_time' column; 
# renames values in program_id column from integers to string names; converts 
# 'start_date', 'end_date', 'created_at', 'updated_at', and 'date_time' from 
# object to datetime format; renames 'name' and 'program_id' columns to 
# 'cohort' and 'programs' respectively
q2_df = p.q2_df_prep(df)

KeyError: "['Unnamed: 0'] not found in axis"

In [ ]:
# returns information on number of non-nulls for each column and their datatype
q2_df.info()

In [ ]:
# shows preview the top of the dataframe
q2_df.head()

## Exploration
- Question 2: Is there a cohort that referred to a lesson significantly more than other cohorts seemed to gloss over?

In [ ]:
# Returns a dataframe with entries that fall within the start and end dates for their respective cohort
q2_df = q2_df[(q2_df['date_time'] >= q2_df['start_date']) & (q2_df['date_time'] <= q2_df['end_date'])]
q2_df.head(3)

In [ ]:
# overall row counts by cohort
q2_df['cohort'].value_counts()

In [ ]:
q2_df['programs'].unique().tolist()  

Dataframe abbreviations:
- q2_fsp = full_stack_php
- q2_fsj = full_stack_java
- q2_ds = data_science
- q2_fep = front_end_programming

In [ ]:
# full stack php dataframe
q2_fsp = q2_df[q2_df.programs == 'full_stack_php']
q2_fsp.head(3)

In [ ]:
# full stack java dataframe
q2_fsj  = q2_df[q2_df.programs == 'full_stack_java']
q2_fsj.head(3)

In [ ]:
# data science dataframe
q2_ds = q2_df[q2_df.programs == 'data_science']
q2_ds.head(3)

In [ ]:
# front end programming dataframe
q2_fep = q2_df[q2_df.programs == 'front_end_programming']
q2_fep.head(3)

In [ ]:
# row counts for full stack php
q2_fsp['cohort'].value_counts()

In [ ]:
# row counts for full stack java
q2_fsj['cohort'].value_counts()

In [ ]:
# row counts for data science
q2_ds['cohort'].value_counts()

In [ ]:
# Row counts for front end programming
q2_fep['cohort'].value_counts()

Quick takeaways
- After removing rows whose "date_time" entry didn't fall within the associated "start_date" & "end_date" for the class, it resulted in the the Full Stack PHP and Front End Programming to return empty dataframes after the original dataframe was divided up by program.
- This indicates that all entries for those programs were either before or after their associated class dates
- The Full Stack Java and Data Science programs are the two remaining dataframes that will be use

In [ ]:
q2_fsj.head(3)

In [ ]:
# Plots histogram of number of entries for all Full Stack Java cohorts
plt.figure(figsize=(12,8))
q2_fsj['cohort'].hist()
plt.xticks(rotation = 90)
plt.title("Histogram of total entries by Full Stack Java Cohort")
plt.show()

In [ ]:
# Shows the number of entries for each unique value in the 'path' column for the Full Stack Java Program
q2_fsj_path_counts = q2_fsj['path'].value_counts().to_frame()
q2_fsj_path_counts

In [ ]:
# Shows a preview of the top of the Data Science program dataframe
q2_ds.head(3)

In [ ]:
# Plots histogram of number of entries for all Full Stack Java cohorts
plt.figure(figsize=(12,8))
q2_ds['cohort'].hist()
plt.xticks(rotation = 90)
plt.title("Histogram of total entries by Data Science Cohort")
plt.show()

In [ ]:
# Assigns a variable to the list of fsj lessons
q2_fsj_lessons = ['javascript-i', 'java-iii', 'html-css', 'java-ii', 'jquery', 
                  'mysql', 'spring', 'slides', 'web-design'] 

In [ ]:
# Shows the number of entries for each unique value in the 'path' column for the Data Science Program
q2_ds_path_counts = q2_ds['path'].value_counts().to_frame()
q2_ds_path_counts

In [ ]:
# Assigns a variable to the list of ds lessons
q2_ds_lessons = ['fundamentals', 'classification', 'sql', 'regression', 'anomaly-detection', 
             'python', 'stats', 'clustering', 'storytelling', 'timeseries', 'nlp', 
             'distributed-ml', 'advanced-topics']

In [ ]:
# filtering the rows where that contain 'fundamentals'
q2_ds_fundamentals = q2_ds[q2_ds['path'].str.contains('fundamentals')]
q2_ds_fundamentals.head(3)

In [ ]:
# filtering the rows where that contain 'classification'
q2_ds_classification = q2_ds[q2_ds['path'].str.contains('classification')]
q2_ds_classification.head(3)

In [ ]:
# filtering the rows where that contain 'sql'
q2_ds_sql = q2_ds[q2_ds['path'].str.contains('sql')]
q2_ds_sql.head(3)

In [ ]:
# filtering the rows where that contain 'regression'
q2_ds_regression = q2_ds[q2_ds['path'].str.contains('regression')]
q2_ds_regression.head(3)

In [ ]:
# filtering the rows where that contain 'anomaly-detection'
q2_ds_anomaly_detection = q2_ds[q2_ds['path'].str.contains('anomaly-detection')]
q2_ds_anomaly_detection.head(3)

In [ ]:
# filtering the rows where that contain 'python'
q2_ds_python = q2_ds[q2_ds['path'].str.contains('python')]
q2_ds_python.head(3)

In [ ]:
# filtering the rows where that contain 'stats'
q2_ds_stats = q2_ds[q2_ds['path'].str.contains('stats')]
q2_ds_stats.head(3)

In [ ]:
# filtering the rows where that contain 'clustering'
q2_ds_clustering = q2_ds[q2_ds['path'].str.contains('clustering')]
q2_ds_clustering.head(3)

In [ ]:
# filtering the rows where that contain 'storytelling'
q2_ds_storytelling = q2_ds[q2_ds['path'].str.contains('storytelling')]
q2_ds_storytelling.head(3)

In [ ]:
# filtering the rows where that contain 'timeseries'
q2_ds_timeseries = q2_ds[q2_ds['path'].str.contains('timeseries')]
q2_ds_timeseries.head(3)

In [ ]:
# filtering the rows where that contain 'nlp'
q2_ds_nlp = q2_ds[q2_ds['path'].str.contains('nlp')]
q2_ds_nlp.head(3)

In [ ]:
# filtering the rows where that contain 'distributed-ml'
q2_ds_distributed_ml = q2_ds[q2_ds['path'].str.contains('distributed-ml')]
q2_ds_distributed_ml.head(3)

In [ ]:
# filtering the rows where that contain 'advanced-topics'
q2_ds_advanced_topics = q2_ds[q2_ds['path'].str.contains('advanced-topics')]
q2_ds_advanced_topics.head(3)

In [ ]:
# Returns the number of rows for the respective dataframs of the different programs
q2_fsp_rows = q2_fsp.shape[0]
print(f'The q2_fsp dataframe has {q2_fsp_rows} rows.')

q2_fsj_rows = q2_fsj.shape[0]
print(f'The q2_fsj dataframe has {q2_fsj_rows} rows.')

q2_ds_rows = q2_ds.shape[0]
print(f'The q2_ds dataframe has {q2_ds_rows} rows.')

q2_fep_rows = q2_fep.shape[0]
print(f'The q2_fep dataframe has {q2_fep_rows} rows.')

In [ ]:
# Returns the number of entries for each of the Full Script Java cohorts
q2_fsj['cohort'].value_counts()

In [ ]:
# Returns the number of entries for each of the Data Science cohorts
q2_ds['cohort'].value_counts()

In [ ]:
# df of values in 'path' column that contain 'javascript-i'
q2_fsj_javascript_i = q2_fsj[q2_fsj['path'].str.contains('javascript-i')]
q2_fsj_javascript_i.head(3)

In [ ]:
# df of values in 'path' column that contain 'java-iii'
q2_fsj_java_iii = q2_fsj[q2_fsj['path'].str.contains('java-iii')]
q2_fsj_java_iii.head(3)

In [ ]:
# df of values in 'path' column that contain 'html-css'
q2_fsj_html_css = q2_fsj[q2_fsj['path'].str.contains('html-css')]
q2_fsj_html_css.head(3)

In [ ]:
# df of values in 'path' column that contain 'java-ii'
q2_fsj_java_ii = q2_fsj[q2_fsj['path'].str.contains('java-ii')]
q2_fsj_java_ii.head(3)

In [ ]:
# df of values in 'path' column that contain 'jquery'
q2_fsj_jquery = q2_fsj[q2_fsj['path'].str.contains('jquery')]
q2_fsj_jquery.head(3)

In [ ]:
# df of values in 'path' column that contain 'mysql'
q2_fsj_mysql = q2_fsj[q2_fsj['path'].str.contains('mysql')]
q2_fsj_mysql.head(3)

In [ ]:
# df of values in 'path' column that contain 'spring'
q2_fsj_spring = q2_fsj[q2_fsj['path'].str.contains('spring')]
q2_fsj_spring.head(3)

In [ ]:
# df of values in 'path' column that contain 'slides'
q2_fsj_slides = q2_fsj[q2_fsj['path'].str.contains('slides')]
q2_fsj_slides.head(3)

In [ ]:
# df of values in 'path' column that contain 'web-design'
q2_fsj_web_design = q2_fsj[q2_fsj['path'].str.contains('web-design')]
q2_fsj_web_design.head(3)

In [ ]:
# list of names of ds lesson dfs
q2_ds_lesson_dfs = [q2_ds_fundamentals, q2_ds_classification, q2_ds_sql,
                    q2_ds_regression, q2_ds_anomaly_detection, q2_ds_python,
                    q2_ds_stats, q2_ds_clustering, q2_ds_storytelling, 
                    q2_ds_timeseries, q2_ds_nlp, q2_ds_distributed_ml, 
                    q2_ds_advanced_topics]

In [ ]:
# list of names of fsj lesson dfs
q2_fsj_lesson_dfs = [q2_fsj_javascript_i, q2_fsj_java_iii, q2_fsj_html_css
                     , q2_fsj_java_ii, q2_fsj_jquery, q2_fsj_mysql, 
                     q2_fsj_spring, q2_fsj_slides, q2_fsj_web_design]

Groupby for Data Science

In [ ]:
# groups the 'cohort' column by the number of corresponding values in the 
# 'path' column in descending order
q2_ds_fundamentals_groupby = q2_ds_fundamentals.groupby('cohort')['path'].count()
q2_ds_fundamentals_groupby

In [ ]:
# assigns name to series
q2_ds_fundamentals_groupby.name = 'fundamentals'

In [ ]:
# groups the 'cohort' column by the number of corresponding values in the 
# 'path' column in descending order
q2_ds_classification_groupby = q2_ds_classification.groupby('cohort')['path'].count()
q2_ds_classification_groupby

In [ ]:
# assigns name to series
q2_ds_classification_groupby.name = 'classification'

In [ ]:
# groups the 'cohort' column by the number of corresponding values in the 
# 'path' column in descending order
q2_ds_sql_groupby = q2_ds_sql.groupby('cohort')['path'].count()
q2_ds_sql_groupby

In [ ]:
# assigns name to series
q2_ds_sql_groupby.name = 'sql'

In [ ]:
# groups the 'cohort' column by the number of corresponding values in the 
# 'path' column in descending order
q2_ds_regression_groupby = q2_ds_regression.groupby('cohort')['path'].count()
q2_ds_regression_groupby

In [ ]:
# assigns name to series
q2_ds_regression_groupby.name = 'regression'

In [ ]:
# groups the 'cohort' column by the number of corresponding values in the 
# 'path' column in descending order
q2_ds_anomaly_detection_groupby = q2_ds_anomaly_detection.groupby('cohort')['path'].count()
q2_ds_anomaly_detection_groupby

In [ ]:
# assigns name to series
q2_ds_anomaly_detection_groupby.name = 'anomaly_detection'

In [ ]:
# groups the 'cohort' column by the number of corresponding values in the 
# 'path' column in descending order
q2_ds_python_groupby = q2_ds_python.groupby('cohort')['path'].count()
q2_ds_python_groupby

In [ ]:
# assigns name to series
q2_ds_python_groupby.name = 'python'

In [ ]:
# groups the 'cohort' column by the number of corresponding values in the 
# 'path' column in descending order
q2_ds_stats_groupby = q2_ds_stats.groupby('cohort')['path'].count()
q2_ds_stats_groupby

In [ ]:
# assigns name to series
q2_ds_stats_groupby.name = 'stats'

In [ ]:
# groups the 'cohort' column by the number of corresponding values in the 
# 'path' column in descending order
q2_ds_clustering_groupby = q2_ds_clustering.groupby('cohort')['path'].count()
q2_ds_clustering_groupby

In [ ]:
# assigns name to series
q2_ds_clustering_groupby.name = 'clustering'

In [ ]:
# groups the 'cohort' column by the number of corresponding values in the 
# 'path' column in descending order
q2_ds_storytelling_groupby = q2_ds_storytelling.groupby('cohort')['path'].count()
q2_ds_storytelling_groupby

In [ ]:
# assigns name to series
q2_ds_storytelling_groupby.name = 'storytelling'

In [ ]:
# groups the 'cohort' column by the number of corresponding values in the 
# 'path' column in descending order
q2_ds_timeseries_groupby = q2_ds_timeseries.groupby('cohort')['path'].count()
q2_ds_timeseries_groupby

In [ ]:
# assigns name to series
q2_ds_timeseries_groupby.name = 'timeseries'

In [ ]:
# groups the 'cohort' column by the number of corresponding values in the 
# 'path' column in descending order
q2_ds_nlp_groupby = q2_ds_nlp.groupby('cohort')['path'].count()
q2_ds_nlp_groupby

In [ ]:
# assigns name to series
q2_ds_nlp_groupby.name = 'nlp'

In [ ]:
# groups the 'cohort' column by the number of corresponding values in the 
# 'path' column in descending order
q2_ds_distributed_ml_groupby = q2_ds_distributed_ml.groupby('cohort')['path'].count()
q2_ds_distributed_ml_groupby

In [ ]:
# assigns name to series
q2_ds_distributed_ml_groupby.name = 'distributed_ml'

In [ ]:
# groups the 'cohort' column by the number of corresponding values in the 
# 'path' column in descending order
q2_ds_advanced_topics_groupby = q2_ds_advanced_topics.groupby('cohort')['path'].count()
q2_ds_advanced_topics_groupby

In [ ]:
# assigns name to series
q2_ds_advanced_topics_groupby.name = 'advanced_topics'

Groupby for Full Stack Java

In [ ]:
# groups the 'cohort' column by the number of corresponding values in the 
# 'path' column in descending order
q2_fsj_javascript_i_groupby = q2_fsj_javascript_i.groupby('cohort')['path'].count()
q2_fsj_javascript_i_groupby

In [ ]:
# assigns name to series
q2_fsj_javascript_i_groupby.name = 'javascript_i'

In [ ]:
# groups the 'cohort' column by the number of corresponding values in the 
# 'path' column in descending order
q2_fsj_java_iii_groupby = q2_fsj_java_iii.groupby('cohort')['path'].count()
q2_fsj_java_iii_groupby

In [ ]:
# assigns name to series
q2_fsj_java_iii_groupby.name = 'fsj_java_iii'

In [ ]:
# groups the 'cohort' column by the number of corresponding values in the 
# 'path' column in descending order
q2_fsj_html_css_groupby = q2_fsj_html_css.groupby('cohort')['path'].count()
q2_fsj_html_css_groupby

In [ ]:
# assigns name to series
q2_fsj_html_css_groupby.name = 'html_css'

In [ ]:
# groups the 'cohort' column by the number of corresponding values in the 
# 'path' column in descending order
q2_fsj_java_ii_groupby = q2_fsj_java_ii.groupby('cohort')['path'].count()
q2_fsj_java_ii_groupby

In [ ]:
# assigns name to series
q2_fsj_java_ii_groupby.name = 'java_ii'

In [ ]:
# groups the 'cohort' column by the number of corresponding values in the 
# 'path' column in descending order
q2_fsj_jquery_groupby = q2_fsj_jquery.groupby('cohort')['path'].count()
q2_fsj_jquery_groupby

In [ ]:
# assigns name to series
q2_fsj_jquery_groupby.name = 'jquery'

In [ ]:
# groups the 'cohort' column by the number of corresponding values in the 
# 'path' column in descending order
q2_fsj_mysql_groupby = q2_fsj_mysql.groupby('cohort')['path'].count()
q2_fsj_mysql_groupby

In [ ]:
# assigns name to series
q2_fsj_mysql_groupby.name = 'mysql'

In [ ]:
# groups the 'cohort' column by the number of corresponding values in the 
# 'path' column in descending order
q2_fsj_spring_groupby = q2_fsj_spring.groupby('cohort')['path'].count()
q2_fsj_spring_groupby

In [ ]:
# assigns name to series
q2_fsj_spring_groupby.name = 'spring'

In [ ]:
# groups the 'cohort' column by the number of corresponding values in the 
# 'path' column in descending order
q2_fsj_slides_groupby = q2_fsj_slides.groupby('cohort')['path'].count()
q2_fsj_slides_groupby

In [ ]:
# assigns name to series
q2_fsj_slides_groupby.name = 'slides'

In [ ]:
# groups the 'cohort' column by the number of corresponding values in the 
# 'path' column in descending order
q2_fsj_web_design_groupby = q2_fsj_web_design.groupby('cohort')['path'].count()
q2_fsj_web_design_groupby

In [ ]:
# assigns name to series
q2_fsj_web_design_groupby.name = 'web_design'

In [ ]:
# list of names of ds series groupbys
q2_ds_lesson_dfs_groupby = [q2_ds_fundamentals_groupby, q2_ds_classification_groupby, q2_ds_sql_groupby,
                    q2_ds_regression_groupby, q2_ds_anomaly_detection_groupby, q2_ds_python_groupby,
                    q2_ds_stats_groupby, q2_ds_clustering_groupby, q2_ds_storytelling_groupby, 
                    q2_ds_timeseries_groupby, q2_ds_nlp_groupby, q2_ds_distributed_ml_groupby, 
                    q2_ds_advanced_topics_groupby]

In [ ]:
q2_ds_combined_lessons = pd.concat(q2_ds_lesson_dfs_groupby, axis=1)
q2_ds_combined_lessons

Stacked barplot of the Full Stack Java cohort's lesson

In [ ]:
# Plots a barplot of number of entries for all Full Stack Java cohorts
plt.figure(figsize=(20, 16))
q2_ds_combined_lessons.plot.bar(stacked=True)
plt.legend(bbox_to_anchor=(1.02, 1), loc='upper left', borderaxespad=0)
plt.xticks(rotation = 90)
plt.title("Lessons Accessed Per Full Stack Java Cohort")
plt.show()

##### The counts for top 2 cohorts for each set of lessons for Data Science

In [ ]:
# prints out the names and numbers of the top 2 cohort for each ds lesson set
for group in q2_ds_lesson_dfs_groupby:
    print(group.nlargest(2))

In [ ]:
# list of names of fsj series groupbys
q2_fsj_lesson_dfs_groupby = [q2_fsj_javascript_i_groupby, q2_fsj_java_iii_groupby, q2_fsj_html_css_groupby
                     , q2_fsj_java_ii_groupby, q2_fsj_jquery_groupby, q2_fsj_mysql_groupby, 
                     q2_fsj_spring_groupby, q2_fsj_slides_groupby, q2_fsj_web_design_groupby]

In [ ]:
q2_fsj_combined_lessons = pd.concat(q2_fsj_lesson_dfs_groupby, axis=1)
q2_fsj_combined_lessons

In [ ]:
# Plots a barplot of number of entries for all Full Stack Java cohorts
plt.figure(figsize=(20, 16))
q2_fsj_combined_lessons.plot.bar(stacked=True)
plt.legend(bbox_to_anchor=(1.02, 1), loc='upper left', borderaxespad=0)
plt.xticks(rotation = 90)
plt.title("Lessons Accessed Per Data Science Cohort")
plt.show()

##### The counts for top 2 cohorts for each set of lessons for Full Stack Java

In [ ]:
# prints out the names and numbers of the top 2 cohort for each fsj lesson set
for group in q2_fsj_lesson_dfs_groupby:
    print(group.nlargest(2))

##### Percentage of top set of lessons per Full Stack Javascript cohort over next highest

In [ ]:
# percentage higher of the max value compared to the second highest
q2_ds_fundamentals_percentage = round(((q2_ds_fundamentals_groupby.sort_values(ascending=False)[0]/q2_ds_fundamentals_groupby.sort_values(ascending=False)[1])-1)*100, 2)
q2_ds_fundamentals_percentage

In [ ]:
# percentage higher of the max value compared to the second highest
q2_ds_classification_percentage = round(((q2_ds_classification_groupby.sort_values(ascending=False)[0]/q2_ds_classification_groupby.sort_values(ascending=False)[1])-1)*100, 2)
q2_ds_classification_percentage

In [ ]:
# percentage higher of the max value compared to the second highest
q2_ds_sql_percentage = round(((q2_ds_sql_groupby.sort_values(ascending=False)[0]/q2_ds_sql_groupby.sort_values(ascending=False)[1])-1)*100, 2)
q2_ds_sql_percentage

In [ ]:
# percentage higher of the max value compared to the second highest
q2_ds_regression_percentage = round(((q2_ds_regression_groupby.sort_values(ascending=False)[0]/q2_ds_regression_groupby.sort_values(ascending=False)[1])-1)*100, 2)
q2_ds_regression_percentage

In [ ]:
# percentage higher of the max value compared to the second highest
q2_ds_anomaly_detection_percentage = round(((q2_ds_anomaly_detection_groupby.sort_values(ascending=False)[0]/q2_ds_anomaly_detection_groupby.sort_values(ascending=False)[1])-1)*100, 2)
q2_ds_anomaly_detection_percentage

In [ ]:
# percentage higher of the max value compared to the second highest
q2_ds_python_percentage = round(((q2_ds_python_groupby.sort_values(ascending=False)[0]/q2_ds_python_groupby.sort_values(ascending=False)[1])-1)*100, 2)
q2_ds_python_percentage

In [ ]:
# percentage higher of the max value compared to the second highest
q2_ds_stats_percentage = round(((q2_ds_stats_groupby.sort_values(ascending=False)[0]/q2_ds_stats_groupby.sort_values(ascending=False)[1])-1)*100, 2)
q2_ds_stats_percentage

In [ ]:
# percentage higher of the max value compared to the second highest
q2_ds_clustering_percentage = round(((q2_ds_clustering_groupby.sort_values(ascending=False)[0]/q2_ds_clustering_groupby.sort_values(ascending=False)[1])-1)*100, 2)
q2_ds_clustering_percentage

In [ ]:
# percentage higher of the max value compared to the second highest
q2_ds_storytelling_percentage = round(((q2_ds_storytelling_groupby.sort_values(ascending=False)[0]/q2_ds_storytelling_groupby.sort_values(ascending=False)[1])-1)*100, 2)
q2_ds_storytelling_percentage

In [ ]:
# percentage higher of the max value compared to the second highest
q2_ds_timeseries_percentage = round(((q2_ds_timeseries_groupby.sort_values(ascending=False)[0]/q2_ds_timeseries_groupby.sort_values(ascending=False)[1])-1)*100, 2)
q2_ds_timeseries_percentage

In [ ]:
# percentage higher of the max value compared to the second highest
q2_ds_nlp_percentage = round(((q2_ds_nlp_groupby.sort_values(ascending=False)[0]/q2_ds_nlp_groupby.sort_values(ascending=False)[1])-1)*100, 2)
q2_ds_nlp_percentage

In [ ]:
# percentage higher of the max value compared to the second highest
q2_ds_distributed_ml_percentage = round(((q2_ds_distributed_ml_groupby.sort_values(ascending=False)[0]/q2_ds_distributed_ml_groupby.sort_values(ascending=False)[1])-1)*100, 2)
q2_ds_distributed_ml_percentage

In [ ]:
# percentage higher of the max value compared to the second highest
q2_ds_advanced_topics_percentage = round(((q2_ds_advanced_topics_groupby.sort_values(ascending=False)[0]/q2_ds_advanced_topics_groupby.sort_values(ascending=False)[1])-1)*100, 2)
q2_ds_advanced_topics_percentage

##### Percentage of top set of lessons per Full Stack Javascript cohort over next highest

In [ ]:
# Percentage higher of the max value compared to the second highest
q2_fsj_javascript_i_percentage = round(((q2_fsj_javascript_i_groupby.sort_values(ascending=False)[0]/q2_fsj_javascript_i_groupby.sort_values(ascending=False)[1])-1)*100, 2)
q2_fsj_javascript_i_percentage

In [ ]:
# Percentage higher of the max value compared to the second highest
q2_fsj_java_iii_percentage = round(((q2_fsj_java_iii_groupby.sort_values(ascending=False)[0]/q2_fsj_java_iii_groupby.sort_values(ascending=False)[1])-1)*100, 2)
q2_fsj_java_iii_percentage

In [ ]:
# Percentage higher of the max value compared to the second highest
q2_fsj_html_css_percentage = round(((q2_fsj_html_css_groupby.sort_values(ascending=False)[0]/q2_fsj_html_css_groupby.sort_values(ascending=False)[1])-1)*100, 2)
q2_fsj_html_css_percentage

In [ ]:
# Percentage higher of the max value compared to the second highest
q2_fsj_java_ii_percentage = round(((q2_fsj_java_ii_groupby.sort_values(ascending=False)[0]/q2_fsj_java_ii_groupby.sort_values(ascending=False)[1])-1)*100, 2)
q2_fsj_java_ii_percentage

In [ ]:
# Percentage higher of the max value compared to the second highest
q2_fsj_jquery_percentage = round(((q2_fsj_jquery_groupby.sort_values(ascending=False)[0]/q2_fsj_jquery_groupby.sort_values(ascending=False)[1])-1)*100, 2)
q2_fsj_jquery_percentage

In [ ]:
# Percentage higher of the max value compared to the second highest
q2_fsj_mysql_percentage = round(((q2_fsj_mysql_groupby.sort_values(ascending=False)[0]/q2_fsj_mysql_groupby.sort_values(ascending=False)[1])-1)*100, 2)
q2_fsj_mysql_percentage

In [ ]:
# Percentage higher of the max value compared to the second highest
q2_fsj_spring_percentage = round(((q2_fsj_spring_groupby.sort_values(ascending=False)[0]/q2_fsj_spring_groupby.sort_values(ascending=False)[1])-1)*100, 2)
q2_fsj_spring_percentage

In [ ]:
# Percentage higher of the max value compared to the second highest
q2_fsj_slides_percentage = round(((q2_fsj_slides_groupby.sort_values(ascending=False)[0]/q2_fsj_slides_groupby.sort_values(ascending=False)[1])-1)*100, 2)
q2_fsj_slides_percentage

In [ ]:
# Percentage higher of the max value compared to the second highest
q2_fsj_web_design_percentage = round(((q2_fsj_web_design_groupby.sort_values(ascending=False)[0]/q2_fsj_web_design_groupby.sort_values(ascending=False)[1])-1)*100, 2)
q2_fsj_web_design_percentage

#### Key Takeaways

The Data Science cohorts with the highest lesson referrals compared to the next highest cohort are as follows:
- Darden
    - 73.60% for Classification
    - 55.56% for SQL
    - 36.70% for Anomaly Detection
- Bayes
    - 36.27% for Advanced Topics

The Full Stack Java cohorts with the highest lesson referrals compared to the next highest cohort are as follows:
- Ceres
    - 42.96% for HTML-CSS
- Jupiter
    - 26.5% for Java II

In [ ]:
# Assigns a variable to a list of the differences in percentages between the top 2 cohorts for each lesson
q2_ds_lesson_percentages = [q2_ds_fundamentals_percentage, q2_ds_classification_percentage, 
                            q2_ds_sql_percentage, q2_ds_regression_percentage, q2_ds_anomaly_detection_percentage,
                            q2_ds_python_percentage, q2_ds_stats_percentage, q2_ds_clustering_percentage, 
                            q2_ds_storytelling_percentage, q2_ds_timeseries_percentage, q2_ds_nlp_percentage, 
                            q2_ds_distributed_ml_percentage, q2_ds_advanced_topics_percentage]

In [ ]:
# Assigns a variable to a list of the differences in percentages between the top 2 cohorts for each lesson
q2_fsj_lesson_percentages = [q2_fsj_javascript_i_percentage, q2_fsj_java_iii_percentage, 
                             q2_fsj_html_css_percentage, q2_fsj_java_ii_percentage, q2_fsj_jquery_percentage,
                             q2_fsj_mysql_percentage, q2_fsj_spring_percentage, q2_fsj_slides_percentage, 
                             q2_fsj_web_design_percentage]

In [ ]:
# Assigns a variable to a series converted from a list of the differences in 
# percentages between the top 2 cohorts for each lesson
ds_upper_bound_and_max = pd.Series(q2_ds_lesson_percentages)

# assigns a name to the series
ds_upper_bound_and_max.name = 'data_science_cohorts'

In [ ]:
# Assigns a variable to a series converted from a list of the differences in 
# percentages between the top 2 cohorts for each lesson
fsj_upper_bound_and_max = pd.Series(q2_fsj_lesson_percentages)

# assigns a name to the series
fsj_upper_bound_and_max.name = 'full_stack_java_cohorts'

In [ ]:
# Gets the upper bound and difference from max of a series based on the InterQuartile Range
# and a multiplier. Default multiplier is 1.5
q2_ds_iqr = e.get_upper_bound_and_difference(ds_upper_bound_and_max)

In [ ]:
# Gets the upper bound and difference from max of a series based on the InterQuartile Range
# and a multiplier. Default multiplier is 1.5
q2_fsj_iqr = e.get_upper_bound_and_difference(fsj_upper_bound_and_max)

#### Key Takeaways

- Using the Interquartile Range Rule the Darden Data Science cohort falls just within the upper bound of 74.85% compared to their 73.60%
- Using the Interquartile Range Rule the Ceres Full Stack Java cohort also falls within the upper bound of 52.66% compared to their 42.96%

### Second Interpretation
 - Using IQR on the whole set instead of top two cohorts

In [ ]:
# Gets the upper bound and its difference from the max of a series based on the 
# InterQuartile Range and a multiplier. Default multiplier is 1.5
for lesson in q2_ds_lesson_dfs_groupby:
    e.get_upper_bound_and_difference(lesson)

In [ ]:
# Gets the upper bound and its difference from the max of a series based on the 
# InterQuartile Range and a multiplier. Default multiplier is 1.5
for group in q2_fsj_lesson_dfs_groupby:
    e.get_upper_bound_and_difference(group)

In [ ]:
# Average of the sum of all values excluding the max
q2_ds_sql_under_max_avg = (q2_ds_sql_groupby.sum()-q2_ds_sql_groupby.max())/(len(q2_fsj_html_css_groupby)-1)

In [ ]:
# Max value for a cohort for these lessons
q2_ds_sql_max = q2_ds_sql_groupby.max()

In [ ]:
# Returns percentage difference of the max value compared to the average of the rest
q2_ds_sql_percentage_difference = round((q2_ds_sql_max/q2_ds_sql_under_max_avg*100), 2)
q2_ds_sql_percentage_difference

#### Key Takeaways

It shows the following Data Science lesson was referred to above the 1.5 IQR Upper Bound:
- SQL at 701.5 above the Upper bound with a 996.28% difference compared to average of rest of cohorts.
- While at 55.56% this Darden cohort may have been lower percentage-wise then its next immediate cohort, Curie, it's a more significant difference in comparison to all other cohorts referring to their respective lessons

It shows the following Full Stack Java lessons were referred to above the 1.5 IQR Upper Bound:
- HTML-CSS for Ceres, at 523.38, a 207.34% difference compared to average of rest of cohorts.
- JQuery for Ceres, at 64.75, a 170.93% difference compared to average of rest of cohorts.
- Slides for Teddy, at 1511.62, a 871.26% difference compared to average of rest of cohorts.
- While at 42.96% the Ceres cohort may have been under the 1.5 IQR percentage-wise when comparing the next highest cohort to the top cohort for each lesson, the Teddy cohort comes out on top when comparing the average of the rest of the cohorts when referring to the "Slides" lessons.

In [ ]:
# Average of the sum of all values excluding the max
q2_fsj_html_css_under_max_avg = (q2_fsj_html_css_groupby.sum()-q2_fsj_html_css_groupby.max())/(len(q2_fsj_html_css_groupby)-1)

In [ ]:
# Max value for a cohort for these lessons
q2_fsj_html_css_max = q2_fsj_html_css_groupby.max()

Final percentage difference for the max vs the average of the rest for the "HTML-CSS" lessons

In [ ]:
# Returns percentage difference of the max value compared to the average of the rest
q2_fsj_html_css_percentage_difference = round((q2_fsj_html_css_max/q2_fsj_html_css_under_max_avg*100), 2)
q2_fsj_html_css_percentage_difference

In [ ]:
# Average of the sum of all values excluding the max
q2_fsj_jquery_under_max_avg = (q2_fsj_jquery_groupby.sum()-q2_fsj_jquery_groupby.max())/(len(q2_fsj_jquery_groupby)-1)

In [ ]:
# max value for a cohort oor these lessons
q2_fsj_jquery_max = q2_fsj_jquery_groupby.max()

Final percentage difference for the max vs the average of the rest for the "JQuery" lessons

In [ ]:
# Returns percentage difference of the max value compared to the average of the rest
q2_fsj_jquery_difference = round((q2_fsj_jquery_max/q2_fsj_jquery_under_max_avg*100), 2)
q2_fsj_jquery_difference

In [ ]:
# Average of the sum of all values excluding the max
q2_fsj_slides_under_max_avg = (q2_fsj_slides_groupby.sum()-q2_fsj_slides_groupby.max())/(len(q2_fsj_slides_groupby)-1)

In [ ]:
# Max value for a cohort for these lessons
q2_fsj_slides_max = q2_fsj_slides_groupby.max()

Final percentage difference for the max vs the average of the rest for the "Slides" lessons

In [ ]:
# Returns percentage difference of the max value compared to the average of the rest
q2_fsj_slides_difference = round((q2_fsj_slides_max/q2_fsj_slides_under_max_avg*100), 2)
q2_fsj_slides_difference

# Q2. Summary of Takeaways
- Question 2. *"Is there a cohort that referred to a lesson significantly more than other cohorts seemed to gloss over?"*
- If applying the Interquartile Range (IQR) Rule to the difference in the percentages between the top cohort and it's runner-up for each set of lessons, none of them fall above the upper bound of the IQR.
- However when using the IQR to for the top cohort in comparison to the rest of the cohorts as a whole then for the **Teddy** cohort from the **Full Stack Java** program beats out the rest with the **"Slides"** lessons, and the **Darden** cohort from the **Data Science** program beats out the rest with the **"SQL"** lessons.
